In [ ]:

import warnings

import numpy as np
import pandas as pd
import swifter
from rdkit import Chem, RDLogger
from rdkit.Chem import rdchem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.MolStandardize import rdMolStandardize

warnings.filterwarnings("ignore")
RDLogger.DisableLog("rdApp.info")


from importlib import reload

import compare_smiles_code
import pandas as pd

reload(compare_smiles_code)

from compare_smiles_code import (
    check_smiles,
    check_smiles_fragments_resonance,
    check_smiles_tmc_resonance,
    count,
    get_agreement_between_multiple_smiles,
    get_agreement_between_smiles,
    process_paralell,
)
from hydrogen_parser import *

import xyz2mol_tm.NBO_to_smiles.mol_utils as mol_utils
from xyz2mol_tm.NBO_to_smiles.mol_utils import strip_stereo

IPythonConsole.molSize = 450, 450
IPythonConsole.drawOptions.addAtomIndices = False
IPythonConsole.drawOptions.useSVG = True
IPythonConsole.drawOptions.minFontSize = 24
# IPythonConsole.drawOptions.legendFraction = 0.2
IPythonConsole.molSize = 500, 500


def print_to_file(line, file_name):
    with open(f"{file_name}", "a") as f:
        f.write(line + "\n")

# Comparing the SMILES obtained with the 3 methods

### Load the smiles

In [ ]:
combined_frame = pd.read_csv("../SMILES_csvs/tmqmg_smiles.csv")


# Strip chiral tags:
combined_frame["smiles_NBO_DFT_xyz"] = combined_frame["smiles_NBO_DFT_xyz"].swifter.apply(mol_utils.strip_stereo)

### Instantiate the masks checking for invalid smiles values

In [ ]:
mask_csd = combined_frame["smiles_CSD_fix"].apply(check_smiles)
mask_tmqmg = combined_frame["smiles_NBO_DFT_xyz"].apply(check_smiles)
mask_huckel = combined_frame["smiles_huckel_DFT_xyz"].apply(check_smiles)

### How many smiles for each method?

In [ ]:
# Aligning the print statements
print(f"CSD smiles:     {len(combined_frame[mask_csd]):>5} / {len(combined_frame):>5}")
print(f"NBO smiles:     {len(combined_frame[mask_tmqmg]):>5} / {len(combined_frame):>5}")
print(f"Huckel smiles:  {len(combined_frame[mask_huckel]):>5} / {len(combined_frame):>5}")
print(f"3 valid SMILES: {len(combined_frame[mask_csd & mask_tmqmg & mask_huckel]):>5} / {len(combined_frame):>5}")

In [ ]:
not_valid_smiles = combined_frame[~(mask_csd | mask_tmqmg | mask_huckel)]

In [ ]:
print(f"{len(not_valid_smiles)} does not have a valid SMILES for either approach")
print("We remove them from the dataframe")
combined_frame.drop(not_valid_smiles.index.values, inplace=True)

### SMILES with incorrect formulas (missing hydrogens)

In [ ]:

mask_csd = combined_frame["smiles_CSD_fix"].apply(check_smiles)
mask_tmqmg = combined_frame["smiles_NBO_DFT_xyz"].apply(check_smiles)
mask_huckel = combined_frame["smiles_huckel_DFT_xyz"].apply(check_smiles)

In [ ]:
df = combined_frame[(mask_csd | mask_tmqmg | mask_huckel)]

In [ ]:
combined_frame.columns

In [ ]:
# Ground truth number of hydrogens from the CSD API formula
df["number_hydrogen_csd_api"] = df["formula_heaviest_fragment"].apply(parse_formula, task="hydrogen")

In [ ]:
# Step 2: Apply the function only to the rows where the mask is True
df["formula_smiles_huckel_DFT_xyz"] = np.nan  # Initialize with NaN
df.loc[mask_huckel, "formula_smiles_huckel_DFT_xyz"] = df.loc[mask_huckel, "smiles_huckel_DFT_xyz"].swifter.apply(get_formula_from_smiles)

df["formula_csd_smiles"] = np.nan  # Initialize with NaN
df.loc[mask_csd, "formula_csd_smiles"] = df.loc[mask_csd, "smiles_CSD_fix"].swifter.apply(get_formula_from_smiles)

df["formula_nbo_smiles"] = np.nan  # Initialize with NaN
df.loc[mask_tmqmg, "formula_nbo_smiles"] = df.loc[mask_tmqmg, "smiles_NBO_DFT_xyz"].swifter.apply(get_formula_from_smiles)

In [ ]:
df["number_hydrogen_huckel_formula"] = df.swifter.apply(
    lambda x: parse_formula(
        x.formula_smiles_huckel_DFT_xyz,
        task="hydrogen",
    ),
    axis=1,
)
df["number_hydrogen_csd_formula"] = df.swifter.apply(
    lambda x: parse_formula(
        x.formula_csd_smiles,
        task="hydrogen",
    ),
    axis=1,
)
df["number_hydrogen_nbo_formula"] = df.swifter.apply(
    lambda x: parse_formula(
        x.formula_nbo_smiles,
        task="hydrogen",
    ),
    axis=1,
)

In [ ]:

csd_arr = df[mask_csd]["number_hydrogen_csd_formula"]
ground_truth = df["number_hydrogen_csd_api"]
nbo_arr = df[mask_tmqmg]["number_hydrogen_nbo_formula"]
huckel_arr = df[mask_huckel]["number_hydrogen_huckel_formula"]

csd_comp = csd_arr != ground_truth[mask_csd]
nbo_comp = nbo_arr != ground_truth[mask_tmqmg]
huckel_comp = huckel_arr != ground_truth[mask_huckel]

print(f"Number of missing hydrogens for each method:\n")
print(f"CSD {sum(csd_comp)}")
print(f"NBO: {sum(nbo_comp)}")
print(f"Huckel: {sum(huckel_comp)}")

In [ ]:
all_wrong_formula = df[(csd_comp & nbo_comp & huckel_comp)]
print(f"{len(all_wrong_formula)} Entries have wrong or missing formula for all methods. We drop them")
df.drop(all_wrong_formula.index.values, inplace=True)
print(len(df))

To consistently compare SMILES, we remove all rows where one of the three methods have a SMILES with incorrect number of hydrogens.

In [ ]:
one_wrong_formula = df[(csd_comp | nbo_comp | huckel_comp)]
print(f"{len(one_wrong_formula)} Entries have wrong or missing formula for at least one of the methods. We drop them")
df.drop(one_wrong_formula.index.values, inplace=True)
print("Remaining entries in the dataset:", len(df))

## Comparing SMILES directly

In [ ]:
mask_csd = combined_frame["smiles_CSD_fix"].apply(check_smiles)
mask_tmqmg = combined_frame["smiles_NBO_DFT_xyz"].apply(check_smiles)
mask_huckel = combined_frame["smiles_huckel_DFT_xyz"].apply(check_smiles)

In [ ]:
all_have_smiles = combined_frame[(mask_csd & mask_tmqmg) & (mask_huckel)]

In [ ]:

r = all_have_smiles.swifter.apply(
    lambda x: get_agreement_between_multiple_smiles(
        x.smiles_CSD_fix,
        x.smiles_NBO_DFT_xyz,
        x.smiles_huckel_DFT_xyz,
    ),
    axis=1,
)

print_to_file(f"Comparing the SMILES of all three methods: {sum(r)}/{len(all_have_smiles)}", "comparison.txt")

In [ ]:
comp1 = combined_frame[mask_csd & mask_tmqmg]

In [ ]:
r = comp1.swifter.apply(
    lambda x: get_agreement_between_smiles(
        x.smiles_CSD_fix,
        x.smiles_NBO_DFT_xyz,
    ),
    axis=1,
)
print_to_file(f"Comparing smiles directly CSD/NBO: {sum(r)}/{len(comp1)}", "comparison.txt")

In [ ]:
comp2 = combined_frame[mask_csd & mask_huckel]

In [ ]:
r = comp2.swifter.apply(
    lambda x: get_agreement_between_smiles(x.smiles_CSD_fix, x.smiles_huckel_DFT_xyz),
    axis=1,
)
print_to_file(f"Comparing smiles directly CSD/Huckel: {sum(r)}/{len(comp2)}", "comparison.txt")

In [ ]:
comp3 = combined_frame[mask_tmqmg & mask_huckel]

In [ ]:

r = comp3.swifter.apply(
    lambda x: get_agreement_between_smiles(x.smiles_NBO_DFT_xyz, x.smiles_huckel_DFT_xyz),
    axis=1,
)
print_to_file(f"Comparing smiles directly NBO/Huckel: {sum(r)}/{len(comp3)}", "comparison.txt")

In [ ]:
sum(r)

## Comparing SMILES with resonance forms

In [ ]:
arguments = [
    (sm1, sm2)
    for sm1, sm2 in zip(
        comp1.smiles_CSD_fix.to_list(),
        comp1.smiles_NBO_DFT_xyz.to_list(),
    )
]
results = process_paralell(check_smiles_tmc_resonance, arguments)
print_to_file(f"Comparing smiles with resonance CSD/NBO: {count(results)}/{len(results)}", "comparison.txt")

In [ ]:
arguments = [(sm1, sm2) for sm1, sm2 in zip(comp2.smiles_CSD_fix.to_list(), comp2.smiles_huckel_DFT_xyz.to_list())]
results = process_paralell(check_smiles_tmc_resonance, arguments)
print_to_file(f"Comparing smiles with resonance CSD/Huckel: {count(results)}/{len(results)}", "comparison.txt")

In [ ]:
arguments = [(sm1, sm2) for sm1, sm2 in zip(comp3.smiles_NBO_DFT_xyz.to_list(), comp3.smiles_huckel_DFT_xyz.to_list())]
results = process_paralell(check_smiles_tmc_resonance, arguments)
print_to_file(f"Comparing smiles with resonance NBO/Huckel: {count(results)}/{len(results)}", "comparison.txt")

## Compare SMILES with metal disconnector and resonance forms

In [ ]:
arguments = [
    (sm1, sm2)
    for sm1, sm2 in zip(
        comp1.smiles_CSD_fix.to_list(),
        comp1.smiles_NBO_DFT_xyz.to_list(),
    )
]

results = process_paralell(check_smiles_fragments_resonance, arguments)
print_to_file(f"Comparing smiles with disconnected resonances CSD/NBO: {count(results)}/{len(results)}", "comparison.txt")

In [ ]:
arguments = [(sm1, sm2) for sm1, sm2 in zip(comp2.smiles_CSD_fix.to_list(), comp2.smiles_huckel_DFT_xyz.to_list())]
results = process_paralell(check_smiles_fragments_resonance, arguments)
print_to_file(f"Comparing smiles with disconnected resonances CSD/Huckel: {count(results)}/{len(results)}", "comparison.txt")

In [ ]:
arguments = [(sm1, sm2) for sm1, sm2 in zip(comp3.smiles_NBO_DFT_xyz.to_list(), comp3.smiles_huckel_DFT_xyz.to_list())]
results = process_paralell(check_smiles_fragments_resonance, arguments)
print_to_file(f"Comparing smiles with disconnected resonances NBO/Huckel: {count(results)}/{len(results)}", "comparison.txt")